# Definições

Todas as bibliotecas utilizadas durante as execuções são carregadas aqui. Como atenção, o módulo de funções do Pytorch é importado apenas para deixar um exemplo de uso do one hot encoder do Pytorch. Tal processamento foi suprimido na versão final, porém o código foi mantido comentado para questão de consulta.

O ponto crítico desta etapa é a definição correta dos diretórios de carregamento de arquivos de dados, bem como os caminhos onde serão salvos os dados processados para processamento durante o treinamento e análise.

In [1]:
import os
import gc
import re
import json
import time
import torch
import warnings
import itertools
import numpy as np
import pandas as pd
import torch.nn as nn
from scipy import stats
from datetime import date
import torch.nn.functional as F
from typing import List, Set, Tuple
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, recall_score, precision_score


DATE_LIMIT = date(2023, 12, 31)
BASE_PATH = os.path.dirname(os.getcwd())
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

USER_DATA_READ=f"{BASE_PATH}/data/users-details-2023.csv"
USER_DATA_SAVE=f"{BASE_PATH}/data/users.parquet"

ANIME_DATA_READ = f"{BASE_PATH}/data/anime-dataset-2023.csv"
ANIME_DATA_SAVE = f"{BASE_PATH}/data/animes.parquet"

SCORE_DATA_READ = f"{BASE_PATH}/data/users-score-2023.csv"
SCORE_DATA_SAVE = f"{BASE_PATH}/data/scores.parquet"

FINAL_DATASET_CUT6_BASIC_USER_DATA = f"{BASE_PATH}/data/scores-cut6-basic.parquet"
FINAL_DATASET_CUT7_BASIC_USER_DATA = f"{BASE_PATH}/data/scores-cut7-basic.parquet"
FINAL_DATASET_CUT8_BASIC_USER_DATA = f"{BASE_PATH}/data/scores-cut8-basic.parquet"

FINAL_DATASET_CUT6_FULL_USER_DATA = f"{BASE_PATH}/data/scores-cut6-full.parquet"
FINAL_DATASET_CUT7_FULL_USER_DATA = f"{BASE_PATH}/data/scores-cut7-full.parquet"
FINAL_DATASET_CUT8_FULL_USER_DATA = f"{BASE_PATH}/data/scores-cut8-full.parquet"

EXPERIMENT_LOG = f"{BASE_PATH}/data/experiment-log.txt"
RESULTS_DIR = f"{BASE_PATH}/data/results"

warnings.filterwarnings("ignore", category=UserWarning)
torch.backends.cudnn.deterministic = True

# Processamento

De modo genérico, o fluxo de processamento de dados contará sempre com uma forma de calcular e exibir informações de estatística descritiva e salvar resultado em arquivos .parquet. Esta super classe de leitura se encarrega de implementar estas funcionalidades básicas e genéricas. O cálculo da estatística descritiva já está programado para lidar de forma diferente com variáveis categóricas e numéricas.

In [2]:
class BaseReader:
    def __init__(self, read_path: str, save_path: str):
        self.file_path = read_path
        self.save_path = save_path

    def to_parquet(self, df: pd.DataFrame) -> None:
        df.to_parquet(self.save_path, index=False)

    def get_stats(self, df: pd.DataFrame, columns: List[str]) -> dict:
        result = dict()
        for c in columns:
            result[c] = {
                "hist": df[c].value_counts(dropna=False).to_dict(),
                "max": df[c].max(skipna=True) if df[c].dtype != "O" else 0,
                "mean": df[c].mean(skipna=True) if df[c].dtype != "O" else 0,
                "median": df[c].median(skipna=True) if df[c].dtype != "O" else 0,
                "min": df[c].min(skipna=True) if df[c].dtype != "O" else 0
            }

        return result
    
    def show_stats(self, result: dict) -> None:
        for column in result.keys():
            # Exibe estatísticas descritivas básicas
            print(f"Estatística descritiva de \"{column}\"")
            print(f"Mínimo: {result[column]["min"]}")
            print(f"Média: {result[column]["mean"]}")
            print(f"Mediana: {result[column]["median"]}")
            print(f"Máximo: {result[column]["max"]}")

            # Avalia a quantidade de nulos
            count = 0
            null = 0
            for k in result[column]["hist"].keys():
                count = count + result[column]["hist"][k]
                if type(k) == float and np.isnan(k):
                    null = result[column]["hist"][k]
            percent = round(null * 100 / count, 2) if count > 0 else 0
            print(f"Quantidade de nulos: {null} ({percent}%)")

            # Exibe uma linha de separação
            print("*" * 40, "\n")

A leitura e processamento da base de usuários define as colunas que serão utilizadas no treinamento, aplicando as seguintes regras:

- Padronização de gênero, atribuindo 0 para MALE e 1 para FEMALE;
- Cálculo da idade a partir da data de nascimento, considerando a data de geração dos dados;
- Padronização dos nomes das colunas, removendo espaços em branco e letras maiúsculas;
- Remoção de linhas com valores nulos para gênero e idade.

In [3]:
class UserReader(BaseReader):
    def __init__(self, read_path: str, save_path: str):
        super().__init__(read_path, save_path)

    def first_process(self) -> pd.DataFrame:
        # Carrega os dados, removendo colunas não utilizadas
        remove_columns = [
            "Username", "Location", "Joined",
            "On Hold", "Plan to Watch", "Rewatched"
        ]
        df = pd.read_csv(self.file_path).drop(remove_columns, axis=1)

        # Faz a troca de gênero definindo Male = 0 e Female = 1
        def clear_gender(value: str) -> int:
            if type(value) != str:
                return None
            return 0 if value.upper() == "MALE" else 1
        df["Gender"] = df["Gender"].apply(clear_gender)

        # Faz a conversão da data de nascimento na idade
        def get_age(birth_date: str | float):
            if type(birth_date) != str:
                return None
            return int((DATE_LIMIT - date.fromisoformat(birth_date.split("T")[0])).days / 365)
        df["age"] = df["Birthday"].apply(get_age)
        df = df.drop(["Birthday"], axis=1)

        # Faz a troca de nomes de colunas
        df = df.rename(columns={
            "Mal ID": "user_id",
            "Gender": "gender",
            "Days Watched": "days_spent_with_anime",
            "Mean Score": "mean_score",
            "Watching": "current_anime_wathing",
            "Completed": "total_anime_watched",
            "Dropped": "dropped_anime",
            "Total Entries": "anime_in_list",
            "Episodes Watched": "episodes_watched"
        })

        # Salva o arquivo limpo
        return df

    def remove_nulls(self, df: pd.DataFrame) -> pd.DataFrame:
        original_rows = len(df)
        df = df.dropna()
        new_rows = len(df)
        percent = round((original_rows - new_rows) * 100 / original_rows, 2)
        print(f"Remoção de {original_rows - new_rows} linhas ({percent}%)")
        return df

Todo o processo de treinamento é encapsulado na seguinte função. As etapas compreendem o carregamento de dados, a aplicação das regras de limpeza descritas na classe de leitura, exibição das estatísticas descritivas das colunas, remoção de nulos e gravação de dados em parquet.

Importante comentar que a remoção de nulos foi implementada após a análise da estatística descritiva dos dados, bem como o ajuste das lógicas de processamento de dados da classe de leitura.

OBS.: Todo o processo é encapsulado dentro de funções para evitar consumo exagerado de memória, garantido também pela chamada explícita do garbage collector ao final da função para evitar lixo na memória durante a execução deste notebook.

In [4]:
def execute_user_analysis():
    reader = UserReader(USER_DATA_READ, USER_DATA_SAVE)
    df_user = reader.first_process()

    stats = reader.get_stats(
        df_user,
        [
            "gender", "days_spent_with_anime", "mean_score",
            "current_anime_wathing", "total_anime_watched",
            "dropped_anime", "anime_in_list", "episodes_watched", "age"
        ]
    )
    reader.show_stats(stats)

    df_user = reader.remove_nulls(df_user)
    reader.to_parquet(df_user)

In [5]:
# execute_user_analysis()
# gc.collect()

De modo semelhante, a classe de leitura da base de dados de animes também herda da super classe de leitura. O processamento destes dados conta com uma padronização de categorias para o material original, removendo granularidade excessiva, já que, por exemplo, "4-koma mangá" é um tipo de organização de quadros de "mangá", porém isto não altera o caso de se tratar do mesmo tipo de material original, que é "mangá".

A duração na base não é informada de forma numérica, mas como texto e com as indicações de "horas" e "minutos". Para recuperar apenas os dados numéricos é aplicada uma função regular para extrair os valores e salvá-los. Outro caso é que para casos em que o número de episódios não está disponível, o valor "UNKNOWN" é cadastrado. Isto gera a necessidade de aplicar uma regra para substituir este dado por NaN e converter o tipo de dados da coluna para float.

A presença do valor "UNKNOWN" também ocorre na coluna de gêneros, sendo necessário a sua remoção. Porém, diferente de outros casos da substituição por NaN, a coluna gêneros é tratada diferente. Os gêneros são valores de multilabel, isso significa que um mesmo anime pode ter um ou mais gêneros. Para a entrada do modelo, os gêneros são pivotados e seu valores se tornam 1 para o caso do anime possuir o gênero ou 0 para o caso contrário.

A remoção de nulos é feita apenas de material original, duração e quantidade de episódios. Também, todo o processamento aqui descrito foi feito a partir da visualização das estatísticas descritivas.


In [6]:
class AnimeReader(BaseReader):
    def __init__(self, read_path: str, save_path: str):
        super().__init__(read_path, save_path)

    def first_process(self) -> pd.DataFrame:
        # Carrega dados
        df = pd.read_csv(self.file_path)

        # Remove colunas não utilizadas
        use_columns = ["anime_id", "Genres", "Episodes", "Source", "Duration"]
        df = df[use_columns]

        # Faz a conversão do texto de duração para o valor numérico
        def extract_duration(description: str):
            if description.upper() == "UNKNOWN":
                return np.nan
            numbers = re.findall(r"[0-9]+", description)
            if len(numbers) == 2:
                return int(numbers[0]) * 60 + int(numbers[1])
            else:
                return int(numbers[0])
        df["Duration"] = df["Duration"].apply(extract_duration)

        # Converte o número de episódios em números e remove nulos
        df["Episodes"] = df["Episodes"].apply(lambda x: float(x) if x.upper() != "UNKNOWN" else np.nan).astype("float64")

        # Aplica uma padronização nos nomes dos materiais originais
        def standard_source(source: str):
            conv_source = {
                "4-koma manga": "manga",
                "Book": "book",
                "Card game": "game",
                "Game": "game",
                "Light novel": "novel",
                "Manga": "manga",
                "Mixed media": "other",
                "Music": "other",
                "Novel": "novel",
                "Original": "original",
                "Other": "other",
                "Picture book": "other",
                "Radio": "other",
                "Unknown": np.nan,
                "Visual novel": "visual_novel",
                "Web manga": "manga",
                "Web novel": "novel"
            }
            try:
                return conv_source[source]
            except:
                return np.nan
        df["Source"] = df["Source"].apply(standard_source)

        # Resolve nomenclatura de gêneros
        df["Genres"] = df["Genres"].apply(lambda x: np.nan if x == "UNKNOWN" else x)

        # Faz a troca dos nomes das colunas
        df = df.rename(columns={
            "Genres": "genres",
            "Episodes": "episodes",
            "Source": "source",
            "Duration": "duration"
        })

        return df
    
    def remove_nulls(self, df: pd.DataFrame) -> pd.DataFrame:
        return df.dropna(subset=["source", "duration", "episodes"])

A execução é encapsulada em funções, para evitar o gasto de memória com os processamentos intermediários. Assim como no caso anterior, o garbage collector é explicitamente invocado para garantir a não permanência de lixo em memória.

In [7]:
def execute_anime_analysis():
    anime_reader = AnimeReader(ANIME_DATA_READ, ANIME_DATA_SAVE)

    df_anime = anime_reader.first_process()
    stats = anime_reader.get_stats(
        df_anime,
        ["genres", "episodes", "source", "duration"]
    )
    anime_reader.show_stats(stats)
    df_anime = anime_reader.remove_nulls(df_anime)
    anime_reader.to_parquet(df_anime)

In [8]:
# execute_anime_analysis()
# gc.collect()

O leitor de classificações é responsável por criar os *datasets* prontos para o treinamento. Aqui são aplicadas as regras de negócio para corte da pontuação para gerar a variável binária de classificação e a inclusão ou não dos metadados. Ao final, são geradas 6 combinações diferentes de bases de dados, que são salvas em parquet para a utilização durante o treinamento.

Neste momento é aplicado o one hot encoder para a coluna do material original, o encoder para a coluna de gêneros e a remoção das colunas de ID (após o processo de merge de dados).

In [9]:
class ScoreReader(BaseReader):
    def __init__(self, read_path: str, save_path: str, user_path: str, anime_path: str):
        super().__init__(read_path, save_path)
        self.anime_path = anime_path
        self.user_path = user_path

    def make_dataset(self, rating_cut=7, user_merge_mode=1) -> pd.DataFrame:
        # Verifica integridade dos parâmetros
        if rating_cut > 10 or rating_cut < 1:
            raise Exception("O corte da classificação deve ser entre 1 e 10")
        
        if user_merge_mode not in [1, 2]:
            raise Exception("O modo de merge de usuário deve ser 1 ou 2")
        
        # Carrega os dados dos scores, limpando as colunas não utilizadas
        df = pd.read_csv(self.file_path)
        df = df.drop(["Username", "Anime Title"], axis=1)

        # Carrega os dados de usuários e animes
        users = pd.read_parquet(self.user_path)
        animes = pd.read_parquet(self.anime_path)

        # Recupera todos os gêneros possíveis
        genres = [[s.strip() for s in g.split(",")] for g in animes["genres"].values if g is not None]
        genres: Set[str] = set(itertools.chain.from_iterable(genres))

        # Define a função de verificação de gênero
        # Os dados de gêneros são carregados como uma string,
        # com as categorias separadas por vírgula
        def verify_genre(genres: str | None, genre: str) -> int:
            if genres is None:
                return 0
            
            genres = [s.lower().strip() for s in genres.split(",")]
            return 1 if genre.lower() in genres else 0

        # Aplica o encoder para gêneros de animes
        for genre in genres:
            column = f"genre_{"_".join(genre.lower().split(" "))}"
            animes[column] = animes["genres"].apply(lambda x: verify_genre(x, genre))
        animes = animes.drop(["genres"], axis=1)

        # Define um encoder para o material original do anime
        encoder = OneHotEncoder(sparse_output=False, handle_unknown="ignore")
        encoder.fit(animes[["source"]])

        # Atualiza os dados de anime com o encoder de material original
        encoder_df = pd.DataFrame(
            encoder.transform(animes[["source"]]),
            columns=encoder.get_feature_names_out()
        )
        animes = pd.concat((animes, encoder_df), axis=1)
        animes = animes.drop(["source"], axis=1)

        # Executa o merge com os dados de usuários
        # user_merge_mode = 1 faz com que apenas os dados básicos sejam usados
        # user_merge_mode = 2 utiliza todos os dados de usuários
        if user_merge_mode == 1:
            users = users[["user_id", "gender", "age"]]

        if user_merge_mode == 2:
            users = users[["user_id", "gender", "age", "days_spent_with_anime", "total_anime_watched", "dropped_anime", "mean_score"]]
        
        df = df.merge(users, how="inner", on="user_id")

        # Executa o merge com os dados de animes
        df = df.merge(animes, how="inner", on="anime_id")

        # Faz a criação da coluna target
        df["target"] = df["rating"].apply(lambda x: 1 if x > rating_cut else 0)
        df = df.drop(["rating"], axis=1)

        # Finaliza o processo, removendo colunas de ID
        df = df.drop(["user_id", "anime_id"], axis=1)
        return df

A função a seguir implementa a lógica de geração de dados. Novamente, como o dataframe utilizado é relativamente grande, é tomado o cuidado para explicitamente remover a variável da memória, com a sequente chamada do garbage collector para garantir a não permanência de lixo.

In [10]:
def create_datasets():
    result_files = [
        (FINAL_DATASET_CUT6_BASIC_USER_DATA, 6, 1),
        (FINAL_DATASET_CUT7_BASIC_USER_DATA, 7, 1),
        (FINAL_DATASET_CUT8_BASIC_USER_DATA, 8, 1),
        (FINAL_DATASET_CUT6_FULL_USER_DATA, 6, 2),
        (FINAL_DATASET_CUT7_FULL_USER_DATA, 7, 2),
        (FINAL_DATASET_CUT8_FULL_USER_DATA, 8, 2)
    ]

    for save_path, cut, mode in result_files:
        score_reader = ScoreReader(
            SCORE_DATA_READ,
            save_path,
            USER_DATA_SAVE,
            ANIME_DATA_SAVE
        )
        scores = score_reader.make_dataset(rating_cut=cut, user_merge_mode=mode)
        score_reader.to_parquet(scores)

        del scores
        gc.collect()

In [11]:
# create_datasets()
# gc.collect()

# Modelo

Para a execução dos experimentos são utilizados 2 modelos MLP, um com 4 camadas ocultas e outro com 8 camadas ocultas, com o intuito de verificar o comportamento dos dados quando se altera a complexidade do modelo.

In [12]:
class Model4Layers(nn.Module):
    def __init__(self, n_features: int, n_classes=2, n_neurons=16):
        super().__init__()
        self.fc1 = nn.Linear(n_features, n_neurons)
        self.fc2 = nn.Linear(n_neurons, n_neurons)
        self.fc3 = nn.Linear(n_neurons, n_neurons)
        self.fc4 = nn.Linear(n_neurons, n_classes)
        self.activation = nn.ReLU()
        self.out = nn.Softmax()

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.fc4(x)
        x = self.out(x)
        return x
    

class Model8Layers(nn.Module):
    def __init__(self, n_features: int, n_classes=2, n_neurons=16):
        super().__init__()
        self.fc1 = nn.Linear(n_features, n_neurons)
        self.fc2 = nn.Linear(n_neurons, n_neurons)
        self.fc3 = nn.Linear(n_neurons, n_neurons)
        self.fc4 = nn.Linear(n_neurons, n_neurons)
        self.fc5 = nn.Linear(n_neurons, n_neurons)
        self.fc6 = nn.Linear(n_neurons, n_neurons)
        self.fc7 = nn.Linear(n_neurons, n_neurons)
        self.fc8 = nn.Linear(n_neurons, n_classes)
        self.activation = nn.ReLU()
        self.out = nn.Softmax()

    def forward(self, x):
        x = self.fc1(x)
        x = self.activation(x)
        x = self.fc2(x)
        x = self.activation(x)
        x = self.fc3(x)
        x = self.activation(x)
        x = self.fc4(x)
        x = self.activation(x)
        x = self.fc5(x)
        x = self.activation(x)
        x = self.fc6(x)
        x = self.activation(x)
        x = self.fc7(x)
        x = self.activation(x)
        x = self.fc8(x)
        x = self.out(x)
        return x

A classe de gerenciamento encpasula o processo de amostragem de dados, treinamento e avaliação das métricas de desempenho. A amostragem é realizada por dois motivos: reduzir o tempo de processamento e o gasto de memória e garantir certa aleatoriedade aos experimentos, uma vez que não é implementada uma validação cruzada.

Novamente, como os recursos de memória e tempo são escassos para este projeto, a validação cruzada se torna um ofensor junto do tamanho dos dados em memória para realizar o treinamento. Como log, os pesos dos modelos são salvos e as métricas encontradas também, registradas junto do log de execução dos experimentos.

In [13]:
class Manager:
    def __init__(self):
        pass

    def get_dataset(self, read_path: str, sample=0.2):
        # Carrega dados processados
        df = pd.read_parquet(read_path).sample(frac=sample)
        X = df.drop(["target"], axis=1).values
        y = df["target"].values

        # Faz a divisão entre treino e teste
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

        # Aplica a padronização de valores
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)

        # Faz a transformação de numpy array para tensor
        X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
        y_train_tensor = torch.tensor(y_train, dtype=torch.long)
        X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
        y_test_tensor = torch.tensor(y_test, dtype=torch.long)

        # Instancia dataset de tensores
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

        # Instancia loader de tensores
        train_loader = DataLoader(train_dataset, batch_size=1000, shuffle=True)
        test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

        return train_dataset, test_dataset, train_loader, test_loader
    
    def execute(self, train_dataset: DataLoader, train_loader: DataLoader, epochs=100, n_neurons=16, arch=1):
        # Garante consistência da arquitetura
        if arch not in [1, 2]:
            raise Exception("As arquiteturas válidas são 1 e 2")
        
        # Regitra o tempo de início do treinamento
        start_time = time.time()

        # Carrega dados e inicializa modelo
        classes_ = len(train_dataset.tensors[1].unique())
        if arch == 1:
            model = Model4Layers(
                n_features=train_dataset.tensors[0].shape[1],
                n_classes=classes_,
                n_neurons=n_neurons
            )
        elif arch == 2:
            model = Model8Layers(
                n_features=train_dataset.tensors[0].shape[1],
                n_classes=classes_,
                n_neurons=n_neurons
            )

        # Define o modo de otimização
        optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
        criterion = nn.CrossEntropyLoss()

        for epoch in range(0, epochs):
            running_loss = 0.0
            running_corrects = 0

            model.train()
            count_batch = 0
            limit_batch = (train_dataset.tensors[0].shape[0] // train_loader.batch_size) + 1

            for inputs, labels in train_loader:
                percent = round(count_batch * 100 / limit_batch, 2)
                print(f"Epoch {epoch + 1} Batch {count_batch + 1} ({percent}%)", end="\r")
                inputs = inputs
                labels = labels

                # Inicia os gradientes e calcula a predição
                optimizer.zero_grad()
                outputs = model(inputs)
                pred_labels = torch.argmax(outputs, dim=1)

                # Teoricamente, seria preciso passar as labels do dataset para o
                # padrão one hot encoder, porém a camada softmax no modelo já
                # resolve isso.
                # oh_labels = F.one_hot(labels.long())
                # loss = criterion(outputs, torch.reshape(oh_labels, (oh_labels.size()[0], classes_)).float())
                loss = criterion(outputs, labels)

                # Calcula os gradientes e atualiza os pesos
                loss.backward()
                optimizer.step()

                # Fal a atualização das estatísticas de acompanhamento
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(pred_labels == labels.data).item()
                count_batch = count_batch + 1

            # Exibe estatísticas de acompanhamento
            num_samples = len(train_dataset)
            epoch_loss = running_loss / num_samples
            epoch_accuracy = running_corrects / num_samples
            print(f"Epoch {epoch + 1}: Loss {epoch_loss:.3f} Acurácia {epoch_accuracy:.3f}")

        # Calcula o tempo de execução do treinamento
        end_time = time.time()
        train_time = end_time - start_time

        return model, train_time, epoch_loss
    
    def compute_test(self, model: nn.Module, test_loader: DataLoader, train_time: float, train_loss: float) -> dict:
        # Define modelo como avaliação e inicia as listas de labels
        model.eval()
        pred_labels_all = []
        true_labels_all = []

        # Passa pelo loader para cálculo das predições
        for inputs, labels in test_loader:
            inputs = inputs
            outputs = model(inputs)
            pred_labels = torch.argmax(outputs, dim=1)
            pred_labels_all.append(pred_labels)
            true_labels_all.append(labels)

        # Concatena os resultados
        pred_labels = torch.cat(pred_labels_all, dim=0).cpu().numpy()
        true_labels = torch.cat(true_labels_all, dim=0).numpy()

        # Registra dados no dicionário de dados
        return {
            "train_time": train_time,
            "train_loss": train_loss,
            "metrics": {
                "accuracy": (pred_labels == true_labels).mean(),
                "f1-score": f1_score(true_labels, pred_labels, pos_label=1, average="binary"),
                "recall-score": recall_score(true_labels, pred_labels, pos_label=1, average="binary"),
                "precission-score": precision_score(true_labels, pred_labels, pos_label=1, average="binary")
            }
        }

Os experimentos são automatizados por meio desta função, que se aproveita dos logs para recuperar o último estágio de treinamento, garantindo a capacidade de executar os experimentos mesmo que sejam interrompidos uma vez. Isto garante flexibilidade para executar os experimentos em momentos distintos e também certa resistência à falha de máquina por um motivo qualquer.

In [14]:
def execute_experiments():
    # Define os parâmetros dos experimentos
    archs_set = [1, 2]
    neurons_set = [16]
    sample_data = [0.1, 0.2]
    epochs = [10]
    repeat = 5
    data_paths = [
        FINAL_DATASET_CUT6_BASIC_USER_DATA,
        FINAL_DATASET_CUT7_BASIC_USER_DATA,
        FINAL_DATASET_CUT8_BASIC_USER_DATA,
        FINAL_DATASET_CUT6_FULL_USER_DATA,
        FINAL_DATASET_CUT7_FULL_USER_DATA,
        FINAL_DATASET_CUT8_FULL_USER_DATA
    ]

    # Verifica o log de experimentos
    if not os.path.exists(EXPERIMENT_LOG):
        with open(EXPERIMENT_LOG, "w") as file:
            file.write("dataset_type,arch,neurons,sample,epochs,iteration,weight_file,predict_file\n")

    # Função auxiliar: abre o log e verifica registros
    def verify(dataset_type: str, arch: int, neurons: int, sample: float, epochs: int, iteration: int):
        exist = False
        with open(EXPERIMENT_LOG, "r") as file:
            row = file.readline()
            while row:
                row_dataset_type, row_arch, row_neurons, row_sample, row_epochs, row_iteration, _, _ = row.split(",")
                row_params = [row_dataset_type, row_arch, row_neurons, row_sample, row_epochs, row_iteration]
                search_params = [str(dataset_type), str(arch), str(neurons), str(sample), str(epochs), str(iteration)]
                
                if row_params == search_params:
                    exist = True
                    break

                row = file.readline()

        return exist

    for data_path in data_paths:
        # Tipo de dataset utilizado
        dataset_type = data_path.split("/")[-1].split(".")[0]

        for neurons in neurons_set:
            # Quantidade de neurônios das camadas internas

            for sample in sample_data:
                # Porção dos dados fracionados

                for epoch in epochs:
                    # Quantidade de épocas do treinamento

                    for arch in archs_set:
                        # Profundidade da rede

                        for i in range(0, repeat):
                            # Verifica se o experimento já foi executado
                            if verify(dataset_type, arch, neurons, sample, epoch, i):
                                continue

                            # Registra todos os dados do experimento
                            unique_name = int(time.time())
                            weight_path = f"{RESULTS_DIR}/{unique_name}.pth"
                            predict_path = f"{RESULTS_DIR}/{unique_name}.json"
                            experiment_data = [
                                    dataset_type,
                                    str(arch),
                                    str(neurons),
                                    str(sample),
                                    str(epoch),
                                    str(i),
                                    f"{unique_name}.pth",
                                    f"{unique_name}.json"
                                ]

                            # Log de execução
                            print(f"Execução do experimento {",".join(experiment_data[:-2])}".upper())

                            # Repetição do experimento
                            process = Manager()
                            train_dataset, test_dataset, train_loader, test_loader = process.get_dataset(data_path, sample=sample)
                            gc.collect()
                            model, train_time, train_loss = process.execute(train_dataset, train_loader, epochs=epoch, n_neurons=neurons, arch=arch)

                            # Executa o teste do modelo
                            results = process.compute_test(model, test_loader, train_time, train_loss)
                            
                            # Salva o json de métricas
                            with open(predict_path, "w+") as file:
                                file.write(json.dumps(results))

                            # Salva os pesos do modelo
                            torch.save(model.state_dict(), weight_path)

                            # Registra no log
                            with open(EXPERIMENT_LOG, "a") as file:
                                file.write(",".join(experiment_data) + "\n")

                            # Libera memória
                            del train_dataset, test_dataset, train_loader, test_loader
                            gc.collect()
                            print()

In [15]:
# execute_experiments()

# Análise

Como principal necessidade é preciso calcular as médias junto do teste estatístico de média, categorizado por cada fator (tratamento) dos experimentos. Esta função genérica é implementada para garantir esta possibilidade, permitindo que se definas as colunas que caracterizam o fator e a coluna de análise. Isto permite maior limpeza e replicabilidade da análise.

O padrão de análise é sobre o F1-score, porém é possível alterar para qualquer métrica presente no log dos experimentos. Todavia, para simplificações, o F1-score foi escolhido como a métrica utilizada neste trabalho exploratório, justamente por ser uma média harmônica entre revocação e precisão. Já para o teste de média, foi utilizado o test t, já disponível na biblioteca scipy.

In [16]:
def generate_analysis(
    df: pd.DataFrame,
    factor_columns: List[str],
    analysis_column: str,
    metrics_dir: str,
    metric_name="f1-score"
):
    # Instancia o dataset de resposta
    df_result = []

    # Faz a criação dos fatores da execução
    factors: List[List[Tuple[str, int|float|str]]] = []
    values = dict()

    for column in factor_columns:
        values[column] = []
        for value in df[column].unique():
            values[column].append((column, value))

    factors = list(itertools.product(*values.values()))

    # Para cada fator, cria o subset de análise
    for factor_list in factors:
        factor_name = []
        subset = df

        for column, value in factor_list:
            subset = subset.loc[subset[column] == value]
            factor_name.append(f"{column}={str(value)}")
        factor_name = ",".join(factor_name)

        # Encontra a quantidade de elementos da análise
        elements = df[analysis_column].unique()
        register = {"factor": factor_name}

        # Percorre o conjunto filtrado pelo fator
        for _, row in subset.iterrows():

            # Acessa o arquivo de métricas
            with open(f"{metrics_dir}/{row["predict_file"]}") as file:
                result = json.load(file)

            # Verifica qual item de análise será utilizado
            for e in elements:
                if row[analysis_column] == e:
                    if e not in register.keys():
                        register[e] = []
                    register[e].append(result["metrics"][metric_name])
                    break
        
        # Executa o teste de média para cada combinação
        combinations = list(itertools.combinations(elements, 2))
        for column1, column2 in combinations:
            register[f"ttest-{column1}-{column2}"] = stats.ttest_ind(
                register[column1], register[column2]
            ).pvalue

        # Converte as listas para a média
        for e in elements:
            register[e] = np.mean(register[e])

        # Salva os dados no dataset de resultados
        df_result.append(register)

    return pd.DataFrame(df_result)

O conjunto de dados dos experimentos é carregado de modo global, garantindo maior facilidade no uso posterior e, como ele não ocupa muito espaço em memória, não se espera um grande impacto.

In [17]:
df = pd.read_csv(EXPERIMENT_LOG)
df["cut"] = df["dataset_type"].apply(lambda x: x.split("-")[1])
df["features"] = df["dataset_type"].apply(lambda x: x.split("-")[2])

## Questionamento: Existe ganho ao se incluir metadados dos usuários?

Uma das primeiras hipóteses é de que a inclusão dos metadados dos usuários aumentaria a capacidade preditiva do modelo. De fato, na maior parte dos fatores analisados isto ocorre, em alguns de modo mais expressivo e em outros de modo menos expressivo. Porém, o teste de média garante que para todos os fatores as médias são diferentes.

In [18]:
generate_analysis(
    df,
    factor_columns=["arch", "sample", "cut"],
    analysis_column="features",
    metrics_dir=RESULTS_DIR
)

,factor,basic,full,ttest-basic-full
0,"arch=1,sample=0.1,cut=cut6",0.888312,0.895944,1.124163e-07
1,"arch=1,sample=0.1,cut=cut7",0.708368,0.762878,1.958774e-08
2,"arch=1,sample=0.1,cut=cut8",0.311931,0.536893,4.948791e-09
3,"arch=1,sample=0.2,cut=cut6",0.888384,0.896093,5.701522e-10
4,"arch=1,sample=0.2,cut=cut7",0.705443,0.761860,1.809408e-09
5,"arch=1,sample=0.2,cut=cut8",0.309045,0.555534,1.653350e-09
6,"arch=2,sample=0.1,cut=cut6",0.887408,0.896105,9.693076e-08
7,"arch=2,sample=0.1,cut=cut7",0.706057,0.762408,1.088779e-07
8,"arch=2,sample=0.1,cut=cut8",0.280773,0.541882,1.329206e-08
9,"arch=2,sample=0.2,cut=cut6",0.888257,0.896011,1.944084e-08


## Questionamento: A forma como definir o limite entre gostou ou não gostou gera impacto no resultado?

Uma decisão de negócio que se estimava impactar no desempenho do modelo é a forma de se definir a variável alvo binária de gostou ou não gostou da obra. A base original é composta por scores de 1 até 10 e foram utilizadas três regras diferentes para conversão em binária:

- cut6: para valores maiores do que 6, considera-se que o usuário gostou do anime;
- cut7: para valores maiores do que 7, considera-se que o usuário gostou do anime;
- cut8: para valores maiores do que 8, considera-se que o usuário gostou do anime.

Para a regra cut6 foram encontrados os maiores valores, porém esta configuração gera um desbalanceamento das classes, com cerca de 80% dos registros marcados como gostei. Isto gera uma dúvida sobre o impacto do balanceamento na capacidade preditiva do modelo, que precisará ser explorado com mais afinco futuramente.

Importante ainda comentar que, para a regra cut8, que é bem mais exigente sobre o gostar ou não gostar, o modelo apresenta os piores resultados. Novamente tem-se uma evidência que que o MLP proposto não consegue lidar bem com o grande desbalanceamento das classes.

In [19]:
def get_target_dist(read_paths: List[str]) -> None:
    for read_path in read_paths:
        df = pd.read_parquet(read_path)

        print(f"Distribuição da target em {read_path}")
        for target, count in df["target"].value_counts().items():
            percent = round(count * 100 / len(df), 2)
            print(f"{target}: {percent}%")
        print("")

        del df
        gc.collect()

In [20]:
generate_analysis(
    df,
    factor_columns=["arch", "sample", "features"],
    analysis_column="cut",
    metrics_dir=RESULTS_DIR
)

,factor,cut6,cut7,cut8,ttest-cut6-cut7,ttest-cut6-cut8,ttest-cut7-cut8
0,"arch=1,sample=0.1,features=basic",0.888312,0.708368,0.311931,1.207986e-13,7.911281e-14,2.349218e-12
1,"arch=1,sample=0.1,features=full",0.895944,0.762878,0.536893,9.855015e-13,1.485798e-11,7.582376e-10
2,"arch=1,sample=0.2,features=basic",0.888384,0.705443,0.309045,1.720762e-14,1.496629e-12,3.513435e-11
3,"arch=1,sample=0.2,features=full",0.896093,0.761860,0.555534,6.473266e-14,7.305112e-16,1.794684e-13
4,"arch=2,sample=0.1,features=basic",0.887408,0.706057,0.280773,8.150079e-12,1.208087e-11,2.788426e-10
5,"arch=2,sample=0.1,features=full",0.896105,0.762408,0.541882,4.742614e-15,3.858148e-14,2.026072e-12
6,"arch=2,sample=0.2,features=basic",0.888257,0.705468,0.303647,1.802789e-14,2.340404e-15,8.447495e-14
7,"arch=2,sample=0.2,features=full",0.896011,0.762367,0.550261,1.199389e-11,1.643081e-14,7.672061e-12


In [21]:
get_target_dist([
    FINAL_DATASET_CUT6_BASIC_USER_DATA,
    FINAL_DATASET_CUT7_BASIC_USER_DATA,
    FINAL_DATASET_CUT8_BASIC_USER_DATA
])

Distribuição da target em /media/bruno/Arquivos/Desenvolvimento/annproject/data/scores-cut6-basic.parquet
1: 79.86%
0: 20.14%

Distribuição da target em /media/bruno/Arquivos/Desenvolvimento/annproject/data/scores-cut7-basic.parquet
1: 57.71%
0: 42.29%

Distribuição da target em /media/bruno/Arquivos/Desenvolvimento/annproject/data/scores-cut8-basic.parquet
0: 67.5%
1: 32.5%



## Questionamento: O aumento da profundidade do MLP melhora o resultado preditivo?

É visível que a métrica não se altera para as diferentes profundidades, o que pode ser confirmado pelo resultado do teste t, cujo valor p é alto demais para rejeitar a hipótese nula (as médias são iguais.).

Vale comentar sobre os fatores 0 e 2, relativos às regras cut6 e cut8, que estatisticamente podem ter seus resultados classificados como diferentes.

In [22]:
generate_analysis(
    df,
    factor_columns=["sample", "features", "cut"],
    analysis_column="arch",
    metrics_dir=RESULTS_DIR
)

,factor,1,2,ttest-1-2
0,"sample=0.1,features=basic,cut=cut6",0.888312,0.887408,0.016724
1,"sample=0.1,features=basic,cut=cut7",0.708368,0.706057,0.536916
2,"sample=0.1,features=basic,cut=cut8",0.311931,0.280773,0.034157
3,"sample=0.1,features=full,cut=cut6",0.895944,0.896105,0.787576
4,"sample=0.1,features=full,cut=cut7",0.762878,0.762408,0.808731
5,"sample=0.1,features=full,cut=cut8",0.536893,0.541882,0.513080
6,"sample=0.2,features=basic,cut=cut6",0.888384,0.888257,0.553339
7,"sample=0.2,features=basic,cut=cut7",0.705443,0.705468,0.990722
8,"sample=0.2,features=basic,cut=cut8",0.309045,0.303647,0.556095
9,"sample=0.2,features=full,cut=cut6",0.896093,0.896011,0.826883


## Questionamento: Aumentar a quantidade de dados amostrados aumenta o desempenho do modelo?

É comum pensar em aumentar a quantidade de dados de treinamento para aumentar o desempenho do modelo. Como por limitações de memória os experimentos são conduzidos com uma amostragem do conjunto de dados originais, pode-se realizar este tipo de teste para verificar se mais dados fazem o modelo ter maior desempenho.

Novamente, as médias e o teste t não deixam evidências de que podemos considerar que existem diferenças entre os casos de diferentes quantidades de dados utilizados para treinamento. Identifica-se uma exceção nos fatores que utilizam a regra cut7 e cut8 (semelhante ao caso anterior), cujos valores do teste t podem indicar diferença entre as médias.

In [23]:
generate_analysis(
    df,
    factor_columns=["arch", "features", "cut"],
    analysis_column="sample",
    metrics_dir=RESULTS_DIR
)

,factor,0.1,0.2,ttest-0.1-0.2
0,"arch=1,features=basic,cut=cut6",0.888312,0.888384,0.777809
1,"arch=1,features=basic,cut=cut7",0.708368,0.705443,0.241177
2,"arch=1,features=basic,cut=cut8",0.311931,0.309045,0.774331
3,"arch=1,features=full,cut=cut6",0.895944,0.896093,0.733955
4,"arch=1,features=full,cut=cut7",0.762878,0.761860,0.642415
5,"arch=1,features=full,cut=cut8",0.536893,0.555534,0.026259
6,"arch=2,features=basic,cut=cut6",0.887408,0.888257,0.013637
7,"arch=2,features=basic,cut=cut7",0.706057,0.705468,0.867389
8,"arch=2,features=basic,cut=cut8",0.280773,0.303647,0.081461
9,"arch=2,features=full,cut=cut6",0.896105,0.896011,0.864736
